In [1]:
txtFile = "/FileStore/tables/shortStackOverflow.txt"
rdd = sc.textFile(txtFile)

In [2]:
# rdd.take(10)

In [3]:
def isSolved(answer):
    if len(answer) > 0:
        return True
    else:
        return False

In [4]:
def parse(line):
    fields = line.split(',')
    if int(fields[0]) == 1:
        return ( int(fields[1]), (1, 0, isSolved(fields[2]), fields[5]))
    else:
        return ( int(fields[3]), (0, 1))

In [5]:
qaRDD = rdd.map(parse)

In [6]:
qaRDD.take(10)

Out[92]: [(100, (1, 0, True, 'Python')),
 (100, (0, 1)),
 (100, (0, 1)),
 (100, (0, 1)),
 (104, (1, 0, False, 'Python')),
 (104, (0, 1)),
 (104, (0, 1)),
 (200, (1, 0, False, 'C#')),
 (200, (0, 1)),
 (200, (0, 1))]

In [7]:
def reduceTasks(a, b):
  question = a
  if b[0] == 1:
    question = b
  if question[0] == 1:
    return (question[0], question[1]+1, question[2], question[3])
  else:
     return (question[1][0], question[1][1]+1)

In [8]:
print(reduceTasks( (1, 0, False, 'Python'), (0, 1)))
reduceRDD=qaRDD.reduceByKey(reduceTasks)

(1, 1, False, 'Python')

In [9]:
reduceRDD.take(10)

Out[95]: [(100, (1, 3, True, 'Python')),
 (104, (1, 2, False, 'Python')),
 (200, (1, 5, False, 'C#')),
 (206, (1, 2, False, 'C#')),
 (300, (1, 0, False, 'Java')),
 (400, (1, 2, True, 'PHP')),
 (301, (1, 3, True, 'Java')),
 (305, (1, 1, True, 'Java')),
 (307, (1, 2, True, 'Java')),
 (403, (1, 1, True, 'PHP'))]

In [10]:
def reduceLang(line):
  return ((line[1][3], line[1][2]), line[1][1])

In [11]:
langRDD = reduceRDD.map(reduceLang)

In [12]:
langRDD.take(10)

Out[98]: [(('Python', True), 3),
 (('Python', False), 2),
 (('C#', False), 5),
 (('C#', False), 2),
 (('Java', False), 0),
 (('PHP', True), 2),
 (('Java', True), 3),
 (('Java', True), 1),
 (('Java', True), 2),
 (('PHP', True), 1)]

In [13]:
combinedRDD = langRDD.combineByKey(
(lambda value: (value, 1)),
(lambda acc, value: (acc[0] + value, acc[1] +1)),
(lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]))
)

In [14]:
combinedRDD.take(10)

Out[100]: [(('Python', False), (2, 1)),
 (('Java', False), (0, 1)),
 (('Python', True), (3, 1)),
 (('C#', False), (7, 2)),
 (('PHP', True), (3, 2)),
 (('Java', True), (6, 3))]

In [15]:
def mapAvg(a):
  if(a[0][1] == False):
    return (a[0][0], (0, float("nan"), a[1][1], a[1][0] / a[1][1]))
  else:
     return (a[0][0], (a[1][1], a[1][0] / a[1][1], 0, float("nan")))
  

In [16]:
print(mapAvg((('Java', False), (0, 1))))
avrageRDD = combinedRDD.map(mapAvg)

('Java', (0, nan, 1, 0.0))

In [17]:
avrageRDD.take(10)

Out[130]: [('Python', (0, nan, 1, 2.0)),
 ('Java', (0, nan, 1, 0.0)),
 ('Python', (1, 3.0, 0, nan)),
 ('C#', (0, nan, 2, 3.5)),
 ('PHP', (2, 1.5, 0, nan)),
 ('Java', (3, 2.0, 0, nan))]

In [18]:
import numpy as np

In [19]:
def reduceSum(a, b):
  tab=[0, 0, 0, 0]
  for i in range(0, 4): 
    if np.isnan(a[i]):
      tab[i]=b[i]
    elif np.isnan(b[i]):
      tab[i]=a[i]
    else:
      tab[i]=a[i]+b[i]
  return (tab[0],tab[1],tab[2],tab[3])
    
    
    

In [20]:
avgFinalRDD = avrageRDD.reduceByKey(reduceSum)

In [21]:
avgFinalRDD.take(10)

Out[145]: [('Python', (1, 3.0, 1, 2.0)),
 ('Java', (3, 2.0, 1, 0.0)),
 ('PHP', (2, 1.5, 0, nan)),
 ('C#', (0, nan, 2, 3.5))]

In [22]:
sortRDD = avgFinalRDD.sortBy(lambda pair: -pair[1][0])

In [23]:
sortRDD.take(10)

Out[147]: [('Java', (3, 2.0, 1, 0.0)),
 ('PHP', (2, 1.5, 0, nan)),
 ('Python', (1, 3.0, 1, 2.0)),
 ('C#', (0, nan, 2, 3.5))]

In [24]:
sortRDD.collect()

Out[148]: [('Java', (3, 2.0, 1, 0.0)),
 ('PHP', (2, 1.5, 0, nan)),
 ('Python', (1, 3.0, 1, 2.0)),
 ('C#', (0, nan, 2, 3.5))]